In [9]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import  HuggingFaceEmbeddings
import os
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
os.getenv("HF_TOKEN")
llm = ChatGroq(model_name = "Gemma-7b-It",groq_api_key=api_key)
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
review_text =FAISS.load_local("reviewText_store",embeddings,allow_dangerous_deserialization=True)
summary_store =FAISS.load_local("summary_store",embeddings,allow_dangerous_deserialization=True)

In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [11]:
review_retriver = review_text.as_retriever()
summary_retriver = summary_store.as_retriever()

## Retriver Chain(For Review Text)

In [12]:
system_prompt_review_text = (
    "You are an Expert Recommendation System "
    "Use the following pieces of retrived context to recommened "
    "based on user query "
    "Suggest Top 3 Best Product Name Only"
    "Do Not Generate Provided Thing Out Of Provided {context}"
    "Do Not Provide Any Additional Information Out Of {context}"
    "\n\n"
    "{context}"
)

prompt_review_text = ChatPromptTemplate.from_messages(
    [
    ("system",system_prompt_review_text),
    ("human", "{input}"),
    ]
)

question_answer_chain_review = create_stuff_documents_chain(llm,prompt_review_text)
rag_chain_review = create_retrieval_chain(review_retriver,question_answer_chain_review)

# response = rag_chain_review.invoke({"input":"jack stands"})

# response['answer']

## Retriver Chain(For Summary Text)

In [13]:
system_prompt_summary_text = (
    "You are an Expert Recommendation System "
    "Use the following pieces of summary context to recommened "
    "based on user query "
    "Suggest Best Top 3 Review For Product Asked By User From {context}"
    "Do Not Generate Provided Thing Out Of Provided {context}"
    "Do Not Provide Any Additional Information Out Of {context}"
    "\n\n"
    "{context}"
)

prompt_summary_text = ChatPromptTemplate.from_messages(
    [
    ("system",system_prompt_summary_text),
    ("human", "{input}"),
    ]
)

question_answer_chain_summary = create_stuff_documents_chain(llm,prompt_summary_text)
rag_chain_summary = create_retrieval_chain(summary_retriver,question_answer_chain_summary)

# response = rag_chain_summary.invoke({"input":"jack stands"})

# response['answer']

## Final RAG Recommendation System

In [14]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate

def generate_final_prompt(review_response, summary_response, query):
    """Generate the final prompt for the LLMChain"""
    prompt = (
        "You Are An Expert Product Recommender "
        "Provide Top Product Name"
        "Based On {review}, {summary} Only "
        "For The User Query: {query} "
        "Do Not Generate Provided Thing Out Of Provided {review}, {summary}"
        "Do Not Provide Any Additional Information Out Of {review}, {summary}"
        "Provide Name Of Product Given In {review}, {summary} Only No Other Information Required"
    ).format(review=review_response, summary=summary_response, query=query)
    return prompt

def recommendation(query):
    """Provide a product recommendation based on the user query"""
    review_response = rag_chain_review.invoke({"input": query})
    summary_response = rag_chain_summary.invoke({"input": query})
    final_prompt = generate_final_prompt(review_response['answer'], summary_response['answer'], query)
    # print("Final Prompt: ", final_prompt)
    prompt_template = PromptTemplate(input_variables=["query"], template=final_prompt)
    chain = LLMChain(llm=llm, prompt=prompt_template)
    response = chain({"query": query})
    return response['text']

In [15]:
print(recommendation("Car Duster"))

**Product Name:** California Car Duster


In [17]:
print(recommendation("Air Filter"))

- **K&N Air Filter**
- **Fram Air Filter**
- **ACDelco Air Filter**


In [19]:
print(recommendation("LED Lights"))

## Top Product Recommendations Based On Provided Reviews:

**1. LEDGLOW KIT 31mm Festoon 12 LEDs SMD LED Bulb White**

* This product received the highest praise in the provided reviews, with one reviewer calling them "freaking awesome" and praising their effectiveness and quality.


**2. Led Bulbs 31mm Festoon LED Light Bulbs (12-Pack White)**

* This product received two highly positive reviews, with one reviewer calling them the "best LEDs on Amazon." Reviewers praised their quality, functionality, and overall performance.


**3. Led Bulbs 31mm Festoon LED Light Bulbs (6-Pack White)**

* While this product only had one review, it received a positive review that praised its ease of installation and effectiveness.

**Additional Notes:**

* All three recommended products received positive feedback for their quality, effectiveness, and ease of installation.
* The LEDGLOW KIT product seemed to impress reviewers the most, likely due to its combination of effectiveness and awesomeness.
* T